# Recuperação e busca de dados em texto

Neste notebook, iremos mostrar alguns algoritmos e estratégias para a recuperação/busca (*retrieval/search*) de dados em formato de texto.

# Instalação de pacotes

# Retrievers e motores de busca

In [ ]:
!pip install langchain==0.1.7
!pip install langchain_community
!pip install langchain_core
!pip install --upgrade --quiet rank_bm25

  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)


In [ ]:
#Vamos definir textos de teste

texto_1 = "Hoje fui para a Semana da Computação da Unicamp!"
texto_2 = "Acabei de jantar no Restaurante Universitário"
texto_3 = "Eu sempre quis saber mais sobre computadores e lógica de programação!"

textos = [texto_1, texto_2, texto_3]

## *Embeddings* semânticos

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1",
)

doc_result_1 = embedding.embed_documents([texto_1])
doc_result_2 = embedding.embed_documents([texto_2])
doc_result_3 = embedding.embed_documents([texto_3])

print(np.dot(doc_result_1[0], doc_result_2[0]))
print(np.dot(doc_result_1[0], doc_result_3[0]))
print(np.dot(doc_result_2[0], doc_result_3[0]))

0.3467101
0.6043142
0.28587684


## BM25

In [ ]:
from rank_bm25 import BM25Okapi

# Tokenização simples
tokenized_texts = [text.split(" ") for text in textos]

# Inicializando o modelo BM25
bm25 = BM25Okapi(tokenized_texts)

# Realizando uma consulta
query = "Restaurante Universitário"
tokenized_query = query.split(" ")

# Obter scores para cada documento
scores = bm25.get_scores(tokenized_query)
print("Scores BM25:", scores)

# Obter os melhores documentos
top_n = bm25.get_top_n(tokenized_query, textos, n=3)
print("Top N documentos BM25:", top_n)

Scores BM25: [0.        1.1858452 0.       ]
Top N documentos BM25: ['Acabei de jantar no Restaurante Universitário', 'Eu sempre quis saber mais sobre computadores e lógica de programação!', 'Hoje fui para a Semana da Computação da Unicamp!']


## Retriever Híbrido

In [ ]:
# Importar bibliotecas necessárias
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
from rank_bm25 import BM25Okapi

# Definir textos de teste
texto_1 = "Hoje fui para a Semana da Computação da Unicamp!"
texto_2 = "Acabei de jantar no Restaurante Universitário"
texto_3 = "Eu sempre quis saber mais sobre computadores e lógica de programação!"

textos = [texto_1, texto_2, texto_3]

# Inicializar embeddings
embedding = HuggingFaceEmbeddings()

# Calcular embeddings dos documentos
doc_embeddings = embedding.embed_documents(textos)

# Tokenização simples para BM25
tokenized_texts = [text.split(" ") for text in textos]

# Inicializar o modelo BM25
bm25 = BM25Okapi(tokenized_texts)

# Função para calcular similaridade de cosseno
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Função para normalizar scores
def normalize(scores):
    return (scores - np.min(scores)) / (np.max(scores) - np.min(scores))

# Realizar uma consulta
query = "Restaurante Universitário"
tokenized_query = query.split(" ")

# Obter scores BM25 para cada documento
bm25_scores = bm25.get_scores(tokenized_query)

# Calcular embedding da query
query_embedding = embedding.embed_query(query)

# Calcular similaridade de cosseno para cada documento
cosine_similarities = [cosine_similarity(query_embedding, doc_embedding) for doc_embedding in doc_embeddings]

# Normalizar as pontuações
normalized_bm25_scores = normalize(bm25_scores)
normalized_cosine_similarities = normalize(cosine_similarities)

# Combinar as pontuações (aqui, utilizando uma média simples)
combined_scores = (normalized_bm25_scores + normalized_cosine_similarities) / 2

# Ordenar os documentos com base nas pontuações combinadas
sorted_indices = np.argsort(combined_scores)[::-1]
top_n_docs = [textos[i] for i in sorted_indices]

# Imprimir os resultados
print("Scores BM25:", bm25_scores)
print("Similaridades de Cosseno:", cosine_similarities)
print("Pontuações Combinadas:", combined_scores)
print("Top N documentos Híbrido:", top_n_docs)

Scores BM25: [0.        1.1858452 0.       ]
Similaridades de Cosseno: [0.34537986, 0.82160217, 0.29798177]
Pontuações Combinadas: [0.04525997 1.         0.        ]
Documentos Top N após Combinação: ['Acabei de jantar no Restaurante Universitário', 'Hoje fui para a Semana da Computação da Unicamp!', 'Eu sempre quis saber mais sobre computadores e lógica de programação!']
Pontuações de Re-rank: [0.82160217, 0.34537986, 0.29798177]
Top N documentos após Re-rank: ['Acabei de jantar no Restaurante Universitário', 'Hoje fui para a Semana da Computação da Unicamp!', 'Eu sempre quis saber mais sobre computadores e lógica de programação!']
